statistics-resampling online 
============

Interact with Octave and the statistics-resampling package online in Jupyter Notebook with Binder. All commands are interpreted by Octave.  Since this is a [MetaKernel](https://github.com/Calysto/metakernel), a standard set of magics are available.  Help on commands is available using the `%help` magic or using `?` with a command.

Follow the links in the 'Quick start' section of the [README page](https://gnu-octave.github.io/statistics-resampling/readme.html), or on the [Function reference page](https://gnu-octave.github.io/statistics-resampling/function_reference), to find an example of a statistical analysis relevant to you. Copy-and-paste the code into the code cell below, update it with your own data and click the run icon in the toolbar. For the best experience, run this notebook in [JupyterLab](https://mybinder.org/v2/gh/acpennlab/statistics-resampling-online/master?labpath=statistics-resampling.ipynb). 
    
To see a list of other Octave packages already installed, use the command `pkg list`. To load a function called `<package-name>`, use the command `pkg load <package-name>`, being sure to replace `<package-name>` with the name of the package you wish to load. Visit this [webpage](https://gnu-octave.github.io/packages/) to see what packages are available for GNU Octave and how to install them. To preview demonstration number `X` of a function called `<function-name>`, use the command `demo ('<function-name>', X)`. For an example of how to load data from a csv file and analyse it with one of the functions from the statistics resampling package, enter the command `demo ('randtest2', 5)`.
    
You could consider forking and customizing the statistics-resampling-online [GitHub repository](https://github.com/acpennlab/statistics-resampling-online) to document and publish your data analysis project.

In [5]:
# Run this cell first to load the statistics-resampling package
pkg load statistics-resampling
ver
pkg describe -verbose statistics-resampling